In [99]:
'''
Notebook for FIG2B-C - get analytical guidance for desinging the punisher
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from design_guidance_tools import *

cpu


Loading BokehJS ...

In [100]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    par, init_conds)  # load the circuit

In [101]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [102]:
# get the cellular variables in steady state without burden
e, F_r, h, xis, Ps = values_for_analytical(par, ode_with_circuit, init_conds,
                                          circuit_genes, circuit_miscs,
                                          circuit_name2pos,
                                          circuit_F_calc)
# record the cellular variables
cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
            'h': h, # intacellular chlorampenicol concentration
            # burden values
            'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
            'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
            # protein degradation correction factors for the switch protein and the integrase
            'P_switch': Ps['switch'], 'P_int': Ps['int']}

In [103]:
# DEFINE SWITCH PROMOTER FOLD-CHANGE AND COOPERATIVITY RANGES

# cooperativity range
eta_range = np.linspace(1.01, 4, 300)

# fold change range
fc_range = np.logspace(np.log10(1), np.log10(200), 300)

In [104]:
# FOLD CHANGE VS COOPERATIVITY: FIND SWITCHING THRESHOLDS AND GUARANTEED FOLD CHANGES

baseline_range = 1 / fc_range

# get a mesh grid, then flatten its x and y coordinates into a single linear array
eta_mesh, baseline_mesh = np.meshgrid(eta_range, baseline_range)
eta_mesh_ravel = eta_mesh.ravel()
baseline_mesh_ravel = baseline_mesh.ravel()

# specify vmapping axes
par_vmapping_axes = {}
for key in par.keys():
    if (key == 'eta_switch' or key == 'baseline_switch'):
        par_vmapping_axes[key] = 0
    else:
        par_vmapping_axes[key] = None
cellvars_vmapping_axes = {}
for key in cellvars.keys():
    cellvars_vmapping_axes[key] = None

# make a vmappable parameter dictionary
par_for_existence = par.copy()
par_for_existence['eta_switch'] = jnp.array(eta_mesh_ravel)
par_for_existence['baseline_switch'] = jnp.array(baseline_mesh_ravel)

# make the checking function vmappable
vmapped_check_if_threshold_exists = jax.jit(jax.vmap(check_if_threshold_exists,
                                                     in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find for which parameter combinations the switching threshold exists
threshold_exists = np.array(vmapped_check_if_threshold_exists(par_for_existence, cellvars))
# from now on, only consider parameter combinations where the threshold bifurcation point exists
indices_where_threshold_exists = []
for i in range(0, len(threshold_exists)):
    if (threshold_exists[i]):
        indices_where_threshold_exists.append(i)
eta_mesh_ravel_exists = eta_mesh_ravel[indices_where_threshold_exists]
baseline_mesh_ravel_exists = baseline_mesh_ravel[indices_where_threshold_exists]

# make a vmappable parameter dictionary
par_for_threshold_gfchanges = par.copy()
par_for_threshold_gfchanges['eta_switch'] = jnp.array(eta_mesh_ravel_exists)
par_for_threshold_gfchanges['baseline_switch'] = jnp.array(baseline_mesh_ravel_exists)

# make the threshold and guaranteed fold change retrieval function vmappable
vmapped_threshold_gfchanges = jax.jit(jax.vmap(threshold_gfchanges,
                                               in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find switching thresholds and guaranteed fold changes for the parameter combinations where the switching threshold exists
thresholds_gfchanges = np.array(vmapped_threshold_gfchanges(par_for_threshold_gfchanges, cellvars))
xi_thresholds = thresholds_gfchanges[:, 1]
gfchange_intact = thresholds_gfchanges[:, 4]

In [105]:
# FOLD CHANGE VS COOPERATIVITY: FIND BURDEN CONTOURS

# fill the points where no threshold exists with INFS
xi_thresholds_for_contour_ravel = np.zeros(threshold_exists.shape)  # initialise
last_index_in_exist_list = 0
for i in range(0, len(xi_thresholds_for_contour_ravel)):
    if (i == indices_where_threshold_exists[last_index_in_exist_list]):
        xi_thresholds_for_contour_ravel[i] = xi_thresholds[last_index_in_exist_list]
        if (last_index_in_exist_list < len(indices_where_threshold_exists) - 1):
            last_index_in_exist_list += 1
    else:
        xi_thresholds_for_contour_ravel[i] = np.inf
xi_thresholds_for_contour = xi_thresholds_for_contour_ravel.reshape(len(fc_range),
                                                                    len(eta_range)).T

# create a contour generator
threshold_cgen = cgen(x=fc_range, y=eta_range,
                      z=xi_thresholds_for_contour)

# contours to be found: 1) all synth. genes functional; 2) just the CAT and protease genes functional
xi_native_prot_cat = cellvars['xi_a'] + cellvars['xi_r'] + cellvars['xi_cat'] + cellvars['xi_prot']
xi_with_all_genes = xi_native_prot_cat + cellvars['xi_other_genes']
xi_contours = {'values': [xi_with_all_genes, xi_native_prot_cat],
               'legends': ['Burden-\nsome\ngene\npresent', 'Burdensome\ngene mutated'],
               'dashes': ['dashed','solid']}

# find burden contour lines
xi_contours['contour lines'] = []
for i in range(0, len(xi_contours['values'])):
    xi_contours['contour lines'].append(threshold_cgen.lines(xi_contours['values'][i]))

In [106]:
# FOLD CHANGE VS COOPERATIVITY: PLOT

# calculate widths and heights for heatmap rectangles - unintuitive due to log scale
rect_widths_along_x_axis = np.zeros(len(fc_range))
rect_widths_along_x_axis[0] = fc_range[1] - fc_range[0]
for i in range(1, len(fc_range)):
    rect_widths_along_x_axis[i] = ((fc_range[i] - fc_range[i - 1]) -
                                   rect_widths_along_x_axis[i - 1] / 2) * 2
rect_heights_along_y_axis = np.zeros(len(eta_range))
rect_heights_along_y_axis[0] = eta_range[1] - eta_range[0]
for i in range(1, len(eta_range)):
    rect_heights_along_y_axis[i] = ((eta_range[i] - eta_range[i - 1]) - rect_heights_along_y_axis[i - 1] / 2) * 2
rect_widths_ravel_exists = np.zeros(baseline_mesh_ravel_exists.shape)
rect_heights_ravel_exists = np.zeros(eta_mesh_ravel_exists.shape)
for i in range(0, len(baseline_mesh_ravel_exists)):
    baseline_where = np.argwhere(
        baseline_range == baseline_mesh_ravel_exists[i])  # locate the baseline value in the baseline range
    rect_widths_ravel_exists[i] = rect_widths_along_x_axis[baseline_where[0][0]]*1.25
    eta_where = np.argwhere(eta_range == eta_mesh_ravel_exists[i])  # locate the eta value in the eta range
    rect_heights_ravel_exists[i] = rect_heights_along_y_axis[eta_where[0][0]]*1.25

# make a dataframe for the heatmap of guaranteed fold changes
heatmap_df = pd.DataFrame({'eta_s': eta_mesh_ravel_exists, 'reciprocal_baseline_s': 1 / baseline_mesh_ravel_exists,
                           'gfchange_intact': gfchange_intact,
                           'rect_width': rect_widths_ravel_exists, 'rect_height': rect_heights_ravel_exists})

gfchange_intact_figure = bkplot.figure(
        frame_width=240,
        frame_height=180,
        x_axis_label="1/Fsb (maximum-to-baseline transc. rate ratio)",
        y_axis_label="η (cooperativity coefficient)",
        x_range=(min(fc_range), max(fc_range)),
        y_range=(min(eta_range), max(eta_range)),
        x_axis_type="log",
        #title="Integrase activity GF-changes",
        tools='pan,box_zoom,reset,save'
    )

# svg backend
gfchange_intact_figure.output_backend= "svg"
# set x ticks
gfchange_intact_figure.xaxis.ticker = bkmodels.FixedTicker(ticks=[1, 10, 100],minor_ticks=[2, 4, 6, 8, 20, 40, 60, 80])
gfchange_intact_figure.xaxis.formatter = bkmodels.PrintfTickFormatter(format="%d")
# plot the heatmap itself
rect=gfchange_intact_figure.rect(x="reciprocal_baseline_s", y="eta_s", source=heatmap_df,
                                 width='rect_width', height='rect_height',
                                 fill_color=bktransform.log_cmap('gfchange_intact',
                                                                    bkpalettes.Plasma256,
                                                                    low=1,
                                                                    high=max(heatmap_df['gfchange_intact'])),
                                 line_width=0, line_alpha=0)
# add colour bar
gfchange_intact_figure.add_layout(rect.construct_color_bar(
    major_label_text_font_size="8pt",
    ticker=bkmodels.FixedTicker(ticks=[1, 100, 10000, 1000000]),
    formatter=bkmodels.PrintfTickFormatter(format="%e"),
    label_standoff=6,
    border_line_color=None,
    padding=5
), 'right')
# plot the burden contours
for i in range(0, len(xi_contours['values'])):
    for j in range(0, len(xi_contours['contour lines'][i])):
        gfchange_intact_figure.line(xi_contours['contour lines'][i][j][:, 0], xi_contours['contour lines'][i][j][:, 1],
                    line_dash=xi_contours['dashes'][i],
                    #legend_label=xi_contours['legends'][i],
                    line_width=2, line_color='black')

# add and configure the legend
# gfchange_intact_figure.legend.location = "bottom_left"
# gfchange_intact_figure.legend.label_text_font_size = "8pt"
# mark where the point with default parameters lays
gfchange_intact_figure.scatter(marker='x',x=[1/par['baseline_switch']], y=[par['eta_switch']], size=8, color='black', line_width=2)

# set fonts
gfchange_intact_figure.xaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.xaxis.major_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.major_label_text_font_size = "8pt"

# add labels for burden thresholds
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][0][0][-1, 0],
                                 y=xi_contours['contour lines'][0][0][-1, 1],
                                 x_offset=-2, y_offset=-48,
                                 text_align='right',
                                 text=xi_contours['legends'][0],
                                 text_font_size='7pt',
                                 text_color='white'))
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][1][0][-1, 0],
                                 y=xi_contours['contour lines'][1][0][-1, 1],
                                 x_offset=10, y_offset=-30,
                                 text_align='left',
                                 text=xi_contours['legends'][1],
                                 text_font_size='7pt',
                                 text_color='white'))
#         
#  show plot
bkplot.show(gfchange_intact_figure)
bkplot.save(gfchange_intact_figure,'fig2b.html')

/tmp/ipykernel_13652/3805163425.py:99: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bkplot.save(gfchange_intact_figure,'fig2b.html')
/tmp/ipykernel_13652/3805163425.py:99: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  bkplot.save(gfchange_intact_figure,'fig2b.html')


'/mnt/c/Users/ersat/CODE/punisher/Fig2/fig2b.html'

In [107]:
# DEFINE THE RANGES OF SWITCH/INTEGRASE/PROTEASE CONCS. AND PROPORTION OF INDUCER-BOUND SWITCH PROTEINS

# switch gene concentration range
c_range=np.linspace(5,20,75)

# bound (active) fraction range
ac_frac_range=np.linspace(0,1,80)

In [108]:
# CONCENTRATION VS ACTIVE FRACTION: FIND SWITCHING THRESHOLDS AND GUARANTEED FOLD CHANGES

# get a mesh grid, then flatten its x and y coordinates into a single linear array
c_mesh, ac_frac_mesh = np.meshgrid(c_range, ac_frac_range)
c_mesh_ravel = c_mesh.ravel()
ac_frac_mesh_ravel = ac_frac_mesh.ravel()

# specify vmapping axes
par_vmapping_axes = {}
for key in par.keys():
    if (key == 'c_switch' or key == 'p_switch_ac_frac'):#or key == 'c_int'
        par_vmapping_axes[key] = 0
    else:
        par_vmapping_axes[key] = None
cellvars_vmapping_axes = {}
for key in cellvars.keys():
    if(key == 'xi_switch_max' or key == 'xi_int_max'):# or key == 'xi_prot'):
        cellvars_vmapping_axes[key] = 0
    else:
        cellvars_vmapping_axes[key] = None
    
# make a vmappable parameter dictionary
par_for_existence = par.copy()
par_for_existence['c_switch'] = jnp.array(c_mesh_ravel)
# par_for_existence['c_int'] = jnp.array(c_mesh_ravel)
# par_for_existence['c_prot'] = jnp.array(c_mesh_ravel)
par_for_existence['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel)
# make a vmappable cellular variable dictionary
cellvars_for_existence = cellvars.copy()
cellvars_for_existence['xi_switch_max'] = (cellvars['xi_switch_max']/par['c_switch']) * jnp.array(c_mesh_ravel)
cellvars_for_existence['xi_int_max'] = (cellvars['xi_int_max']/par['c_switch']) * jnp.array(c_mesh_ravel)
# cellvars_for_existence['xi_prot'] = (cellvars['xi_prot']/par['c_prot']) * jnp.array(c_mesh_ravel)

# make the checking function vmappable
vmapped_check_if_threshold_exists = jax.jit(jax.vmap(check_if_threshold_exists,
                                                     in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find for which parameter combinations the switching threshold exists
threshold_exists = np.array(vmapped_check_if_threshold_exists(par_for_existence, cellvars_for_existence))

# from now on, only consider parameter combinations where the threshold bifurcation point exists
indices_where_threshold_exists = []
for i in range(0, len(threshold_exists)):
    if (threshold_exists[i]):
        indices_where_threshold_exists.append(i)
c_mesh_ravel_exists = c_mesh_ravel[indices_where_threshold_exists]
ac_frac_mesh_ravel_exists = ac_frac_mesh_ravel[indices_where_threshold_exists]

# make a vmappable parameter dictionary
par_for_threshold_gfchanges = par.copy()
par_for_threshold_gfchanges['c_switch'] = jnp.array(c_mesh_ravel_exists)
# par_for_threshold_gfchanges['c_int'] = jnp.array(c_mesh_ravel_exists)
# par_for_threshold_gfchanges['c_prot'] = jnp.array(c_mesh_ravel_exists)
par_for_threshold_gfchanges['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel_exists)
# make a vmappable cellular variable dictionary
cellvars_for_threshold_gfchanges = cellvars.copy()
cellvars_for_threshold_gfchanges['xi_switch_max'] = (cellvars['xi_switch_max']/par['c_switch']) * jnp.array(c_mesh_ravel_exists)
cellvars_for_threshold_gfchanges['xi_int_max'] = (cellvars['xi_int_max']/par['c_switch']) * jnp.array(c_mesh_ravel_exists)
# cellvars_for_threshold_gfchanges['xi_prot'] = (cellvars['xi_prot']/par['c_prot']) * jnp.array(c_mesh_ravel_exists)

# make the threshold and guaranteed fold change retrieval function vmappable
vmapped_threshold_gfchanges = jax.jit(jax.vmap(threshold_gfchanges,
                                               in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find switching thresholds and guaranteed fold changes for the parameter combinations where the switching threshold exists
thresholds_gfchanges = np.array(vmapped_threshold_gfchanges(par_for_threshold_gfchanges, cellvars_for_threshold_gfchanges))
xi_thresholds = thresholds_gfchanges[:, 1]
gfchange_intact = thresholds_gfchanges[:, 4]

In [109]:
# CONCENTRATION VS ACTIVE FRACTION: FIND BURDEN CONTOURS

# fill the points where no threshold exists with INFS
xi_thresholds_for_contour_ravel = np.zeros(threshold_exists.shape)  # initialise
last_index_in_exist_list = 0
for i in range(0, len(xi_thresholds_for_contour_ravel)):
    if (i == indices_where_threshold_exists[last_index_in_exist_list]):
        xi_thresholds_for_contour_ravel[i] = xi_thresholds[last_index_in_exist_list]
        if (last_index_in_exist_list < len(indices_where_threshold_exists) - 1):
            last_index_in_exist_list += 1
    else:
        xi_thresholds_for_contour_ravel[i] = np.inf
xi_thresholds_for_contour = xi_thresholds_for_contour_ravel.reshape(len(ac_frac_range),
                                                                    len(c_range)).T

# create a contour generator
threshold_cgen = cgen(x=ac_frac_range, y=c_range,
                      z=xi_thresholds_for_contour)

# contours to be found: 1) all synth. genes functional; 2) just the CAT and protease genes functional
xi_native_prot_cat = cellvars['xi_a'] + cellvars['xi_r'] + cellvars['xi_cat'] + cellvars['xi_prot']
xi_with_all_genes = xi_native_prot_cat + cellvars['xi_other_genes']
xi_contours = {'values': [xi_with_all_genes, xi_native_prot_cat],
               'legends': ['Burdensome\ngene present', 'Burdensome\ngene mutated'],
               'dashes': ['dashed','solid']}

# find burden contour lines
xi_contours['contour lines'] = []
for i in range(0, len(xi_contours['values'])):
    xi_contours['contour lines'].append(threshold_cgen.lines(xi_contours['values'][i]))

In [111]:
# CONCENTRATIONS VS ACTIVE FRACTIONS: PLOT

rect_widths_along_x_axis = np.zeros(len(ac_frac_range))
rect_widths_along_x_axis[0] = ac_frac_range[1] - ac_frac_range[0]
for i in range(1, len(ac_frac_range)):
    rect_widths_along_x_axis[i] = ((ac_frac_range[i] - ac_frac_range[i - 1]) -
                                   rect_widths_along_x_axis[i - 1] / 2) * 2
rect_heights_along_y_axis = np.zeros(len(c_range))
rect_heights_along_y_axis[0] = c_range[1] - c_range[0]
for i in range(1, len(c_range)):
    rect_heights_along_y_axis[i] = ((c_range[i] - c_range[i - 1]) - rect_heights_along_y_axis[i - 1] / 2) * 2
rect_widths_ravel_exists = np.zeros(ac_frac_mesh_ravel_exists.shape)
rect_heights_ravel_exists = np.zeros(c_mesh_ravel_exists.shape)
for i in range(0, len(ac_frac_mesh_ravel_exists)):
    baseline_where = np.argwhere(
        ac_frac_range == ac_frac_mesh_ravel_exists[i])  # locate the baseline value in the baseline range
    rect_widths_ravel_exists[i] = rect_widths_along_x_axis[baseline_where[0][0]]*1.25
    eta_where = np.argwhere(c_range == c_mesh_ravel_exists[i])  # locate the eta value in the eta range
    rect_heights_ravel_exists[i] = rect_heights_along_y_axis[eta_where[0][0]]*1.25

# make a dataframe for the heatmap of guaranteed fold changes
heatmap_df = pd.DataFrame({'c_s=c_int': c_mesh_ravel_exists, 'I': ac_frac_mesh_ravel_exists,
                           'gfchange_intact': gfchange_intact,
                           'rect_width': rect_widths_ravel_exists, 'rect_height': rect_heights_ravel_exists})


gfchange_intact_figure = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="I (share of switch proteins bound by inducer)",
    y_axis_label="cs=cint (switch and integrase gene conc.), nM",
    x_range=(min(ac_frac_range), max(ac_frac_range)),
    y_range=(min(c_range), max(c_range)),
    #title="Integrase activity GF-change",
    tools='pan,box_zoom,reset,save'
)
# svg backend
gfchange_intact_figure.output_backend= "svg"
# plot the heatmap
rects = gfchange_intact_figure.rect(x="I", y="c_s=c_int", source=heatmap_df,
                                 width='rect_width', height='rect_height',
                                 fill_color=bktransform.log_cmap('gfchange_intact',
                                                                    bkpalettes.Plasma256,
                                                                    low=1,
                                                                    high=max(heatmap_df['gfchange_intact'])),
                                 line_width=0,line_alpha=0)
# add colour bar
gfchange_intact_figure.add_layout(rects.construct_color_bar(
    major_label_text_font_size="8pt",
    ticker=bkmodels.FixedTicker(ticks=[1, 10, 100, 1000, 10000]),
    formatter=bkmodels.PrintfTickFormatter(format="%e"),
    label_standoff=6,
    border_line_color=None,
    padding=5
), 'right')

# mark the line of same DNA concentrations as the default parameter
gfchange_intact_figure.line(x=[min(ac_frac_range), max(ac_frac_range)], y=[par['c_switch'], par['c_switch']],
            line_width=2, line_color='white', line_dash='solid')

# plot the burden contours
for i in range(0,len(xi_contours['values'])):
    for j in range(0,len(xi_contours['contour lines'][i])):
        gfchange_intact_figure.line(xi_contours['contour lines'][i][j][:, 0], xi_contours['contour lines'][i][j][:, 1],
                    line_dash=xi_contours['dashes'][i],
                    #legend_label=xi_contours['legends'][i],
                    line_width=2, line_color='black')

# add and configure the legend
# gfchange_intact_figure.legend.location = "bottom_left"
# gfchange_intact_figure.legend.label_text_font_size = "8pt"

# mark where the point with default parameters lays
gfchange_intact_figure.scatter(marker='x',x=[par['p_switch_ac_frac']], y=[par['c_switch']], size=10, color='black',line_width=4)

# add burden contour labels
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][0][0][0, 0],
                                 y=xi_contours['contour lines'][0][0][0, 1],
                                 x_offset=20, y_offset=-30,
                                 text_align='left',
                                 text=xi_contours['legends'][0],
                                 text_font_size='8pt',
                                 text_color='black'))
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][1][0][0, 0],
                                 y=xi_contours['contour lines'][1][0][0, 1],
                                 x_offset=0, y_offset=-30,
                                 text_align='right',
                                 text=xi_contours['legends'][1],
                                 text_font_size='8pt',
                                 text_color='black'))

# font size
gfchange_intact_figure.xaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.xaxis.major_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.major_label_text_font_size = "8pt"

# show plot
bkplot.show(gfchange_intact_figure)
bkplot.save(gfchange_intact_figure,'Fig2C.html')

/tmp/ipykernel_13652/3917275325.py:100: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bkplot.save(gfchange_intact_figure,'Fig2C.html')
/tmp/ipykernel_13652/3917275325.py:100: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  bkplot.save(gfchange_intact_figure,'Fig2C.html')


'/mnt/c/Users/ersat/CODE/punisher/Fig2/Fig2C.html'